In [ ]:
!pip install -q gradio>=4.0.0 transformers>=4.35.0 torch sentencepiece accelerate protobuf>=3.20.0 peft>=0.6.0 bitsandbytes>=0.41.0

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import warnings

warnings.filterwarnings('ignore')

# --- 모델 설정 ---
# 팀에서 파인튜닝하여 Hugging Face에 업로드한 HyperCLOVA 모델 사용
BASE_MODEL_NAME = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"
LORA_MODEL_NAME = "91veMe4Plus-Project/hyperclova-deobfuscation-lora-with-10k-datasets"

tokenizer = None
model = None
device = None

print("환경 설정 및 모델 로딩을 시작합니다...")

try:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")

    # 양자화 설정 (메모리 절약)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    print(f"베이스 모델 ({BASE_MODEL_NAME}) 로딩 중...")
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto"
    )

    print(f"LoRA 어댑터 ({LORA_MODEL_NAME}) 로딩 중...")
    # LoRA 어댑터 적용
    model = PeftModel.from_pretrained(base_model, LORA_MODEL_NAME)

    print(f"토크나이저 ({LORA_MODEL_NAME}) 로딩 중...")
    # 토크나이저 로드 (LoRA 모델에서 사용한 토크나이저)
    tokenizer = AutoTokenizer.from_pretrained(LORA_MODEL_NAME)

    # pad_token 설정 (필요시)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model.eval() # 추론 모드로 설정
    print(f"모델 및 토크나이저 로딩 완료. Device: {device}")

except Exception as e:
    print(f"모델 로딩 중 심각한 오류 발생: {e}")
    print("Gradio 앱이 정상적으로 작동하지 않을 수 있습니다. 모델 이름 및 인터넷 연결을 확인해주세요.")
    # 오류 발생 시 model이나 tokenizer가 None으로 유지되어 아래 함수에서 처리됩니다.

def deobfuscate_interface(obfuscated_text):
    """
    팀에서 파인튜닝한 HyperCLOVA 모델을 사용하여 텍스트를 비난독화합니다.
    """
    if not model or not tokenizer:
        return "오류: 모델 또는 토크나이저가 로드되지 않았습니다. Colab 셀 실행 로그를 확인해주세요."

    if not obfuscated_text.strip():
        return "난독화된 텍스트를 입력해주세요."

    # HyperCLOVA 파인튜닝 모델에 적합한 프롬프트 형식 (learning_rate_hyperclova_deobfuscation_finetuning.ipynb와 동일)
    prompt = f"### 지시사항:\n다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.\n\n난독화된 텍스트: {obfuscated_text}\n\n### 응답:\n"

    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            # HyperCLOVA 모델에 최적화된 생성 파라미터
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        # 생성된 전체 텍스트에서 입력 프롬프트 부분 제거
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 응답 부분만 추출 (원본 노트북과 동일한 방식)
        if "### 응답:" in generated_text:
            response = generated_text.split("### 응답:")[1].strip()
        else:
            # 프롬프트 길이만큼 제거하는 fallback
            response = generated_text[len(prompt):].strip()

        return response

    except Exception as e:
        print(f"텍스트 생성 중 오류 발생: {e}")
        return f"오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정
iface = gr.Interface(
    fn=deobfuscate_interface,
    inputs=gr.Textbox(
        label="난독화된 한국어 텍스트",
        placeholder="난독화된 텍스트를 입력하세요...",
        lines=5
    ),
    outputs=gr.Textbox(
        label="복원된 원본 텍스트",
        lines=5
    ),
    title="HyperCLOVA 한국어 텍스트 비난독화 (10k 데이터셋)", # 새로운 모델명 반영
    description="팀에서 10k 데이터셋으로 파인튜닝한 `91veMe4Plus-Project/hyperclova-deobfuscation-lora-with-10k-datasets` 모델을 사용하여 난독화된 한국어 텍스트를 원본 텍스트로 복원합니다.", # 새로운 모델 설명
    examples=[ # 예제는 동일한 작업이므로 유지
        ["안녀하쎼요, 반갑쏘니댜!"],
        ["오늬 날씨갸 맆이 좆네욘."],
        ["한큿어 쳬연어 처륄예 댕햔 연귝을 해보갰습닏댜."]
    ]
)

if __name__ == '__main__':
    print("Gradio 인터페이스를 시작합니다...")
    # share=True로 설정하여 외부 접속 링크 생성
    iface.launch(debug=True, share=True)